In [76]:
import os
import pickle
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

from sklearn.metrics.pairwise import cosine_similarity
root_folder = '/home/jixian/github/suchka-ai/output/faces/Jessy Jones/temp'

cluster_threshold = 0.65

def cluster_embeddings(embeddings, threshold):
  groups = []
  data = embeddings
  keys = list(data.keys())
  visited = set()
  m = cosine_similarity(list(data.values()))
  for i in range(len(m)):
    if i in visited: continue
    # print(i, keys[i].replace(' ','\ '))
    group = dict()
    group[keys[i]] = data[keys[i]]
    for j in range(len(m)):
      if j in visited: continue
      if i == j: continue    
      similarity = m[i][j]
      if similarity >= threshold:
        visited.add(i)
        visited.add(j)
        group[keys[j]] = data[keys[j]]
        # print(j, similarity,keys[j].replace(' ','\ '))
    if len(group) > 1: groups.append(group)
  return groups


folders = [f.path for f in os.scandir(root_folder) if f.is_dir()]
data = dict()
for data_folder in folders:
  gid = data_folder.split('/')[-1]
  data_file = f'{data_folder}/embeddings.pkl'
  with open(data_file, 'rb') as f:
    embeddings = pickle.loads(f.read())
    data[gid] = cluster_embeddings(embeddings, cluster_threshold)


In [84]:
common_face_threshold = 0.65

def detected_common_face(data, common_face_threshold):
  common_face = dict()
  pairs = dict()
  for gid in data.keys():  
    other_galleries_gids = [g for g in data.keys() if g != gid]
    for fid in range(len(data[gid])):
      face_id = f'{gid}/{fid}'
      for gid2 in other_galleries_gids:
        for fid2 in range(len(data[gid2])):
          other_face_id = f'{gid2}/{fid2}'
          key = f'{face_id}-{other_face_id}'
          key2 = f'{other_face_id}-{face_id}'
          value = [gid,fid,gid2,fid2]
          if key not in pairs.keys() and key2 not in pairs.keys(): pairs[key] = value
    
  for key, value in pairs.items():  
    face_embeddings = list(data[value[0]][value[1]].values())
    face2_embeddings = list(data[value[2]][value[3]].values())
    similarity = np.mean(cosine_similarity(face_embeddings, face2_embeddings))    
    if similarity >= common_face_threshold:    
      common_face.update(data[value[0]][value[1]])
      common_face.update(data[value[2]][value[3]])      
      break
  return common_face

common_face = detected_common_face(data, common_face_threshold)
print(len(common_face))


15
/home/jixian/github/suchka-ai/output/faces/Jessy Jones/temp/20840305/7c80d0c43a776b4858c8da3cc6359e3c2295ed4f_1.jpg [0.01834561675786972, -0.0005670860409736633, 0.0017192444065585732, 0.01497705653309822, 0.01135948020964861, 0.014239189215004444, -0.004465784877538681, 0.01150246150791645, 0.004020725376904011, 0.013840403407812119, 0.006595028564333916, 0.014313220046460629, 0.01759580709040165, 0.01897832378745079, 0.012079897336661816, 0.03089156560599804, 0.030654016882181168, 0.008042379282414913, 0.0029765572398900986, -0.012617673724889755, 0.015988793224096298, 0.02832241915166378, 0.013132533058524132, 0.009365055710077286, 0.006081013008952141, -0.0007764259353280067, 0.002787591889500618, 0.002837104257196188, 0.020362282171845436, 0.02349969744682312, 0.023808130994439125, 0.025525208562612534, -0.0058760023675858974, 0.014167879708111286, 0.030182858929038048, 0.014892106875777245, 0.0005257133161649108, -0.018216274678707123, 0.030518993735313416, 0.02357479929924011

In [93]:
import shutil
def save_common_face(data, output_path, src_gallery_path):
  embeddings = dict()
  embeddings_file = f'{output_path}/embeddings.pickle'
    
  for f,v in data.items():
    filename = os.path.basename(f).split('_')[0]+'.jpg'    
    gid = f.split('/')[-2]
    embedding_key = f'{src_gallery_path}/{gid}/{filename}'    
    shutil.copy(f, f'{output_path}/{filename}')    
    embeddings[embedding_key] = v
  # ceate embedding file
  with open(embeddings_file, 'wb') as f:
    f.write(pickle.dumps(embeddings))

save_common_face(common_face, '/home/jixian/github/suchka-ai/output/faces/Jessy Jones', 'output/pornpics')

['Jessy Jones']
